# Задача - посчитать cips

## Принесём сюда лучшую модель с первого задания

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.metrics import log_loss
from scipy.special import logit
from scipy.stats import norm

In [2]:
df = pd.read_csv('../data/data.csv')
df.head()

,date_time,zone_id,banner_id,oaid_hash,campaign_clicks,os_id,country_id,banner_id0,rate0,g0,coeff_sum0,banner_id1,rate1,g1,coeff_sum1,impressions,clicks
0,2021-09-27 00:01:30.000000,0,0,5664530014561852622,0,0,0,1240,0.067,0.035016,-7.268846,0,0.010,0.049516,-5.369901,1,1
1,2021-09-26 22:54:49.000000,1,1,5186611064559013950,0,0,1,1,0.002,0.054298,-2.657477,269,0.004,0.031942,-4.449220,1,1
2,2021-09-26 23:57:20.000000,2,2,2215519569292448030,3,0,0,2,0.014,0.014096,-3.824875,21,0.014,0.014906,-3.939309,1,1
3,2021-09-27 00:04:30.000000,3,3,6262169206735077204,0,1,1,3,0.012,0.015232,-3.461357,99,0.006,0.050671,-3.418403,1,1
4,2021-09-27 00:06:21.000000,4,4,4778985830203613115,0,1,0,4,0.019,0.051265,-4.009026,11464230,6.790,0.032005,-2.828797,1,1


In [3]:
df.date_time = pd.to_datetime(df.date_time, format='%Y-%m-%d %H:%M:%S.%f')
df['date'] = df.date_time.dt.date
df = df.sort_values(by='date_time')

In [4]:
train = df[df.date <= datetime.strptime('2021-09-30', '%Y-%m-%d').date()]
val = df[df.date == datetime.strptime('2021-10-01', '%Y-%m-%d').date()]
test = df[df.date == datetime.strptime('2021-10-02', '%Y-%m-%d').date()]

In [5]:
train_slice = len(train)
val_slice = len(val)
test_slice = len(test)

In [6]:
cols = list(set(df.columns).difference(['oaid_hash', 'banner_id0', 'banner_id1', 'rate0', 'rate1', 'g0', 'g1', 'coeff_sum0', 'coeff_sum1']))

In [7]:
df[cols].head()

,date_time,date,country_id,os_id,zone_id,clicks,campaign_clicks,banner_id,impressions
1390198,2021-09-01 00:02:49,2021-09-01,7,0,30,0,0,596,1
5041415,2021-09-26 00:00:00,2021-09-26,0,3,41,0,1,29,1
1442602,2021-09-26 00:00:00,2021-09-26,15,2,1,0,2,188,1
7232498,2021-09-26 00:00:00,2021-09-26,5,2,17,0,2,52,1
14938691,2021-09-26 00:00:00,2021-09-26,13,4,47,0,1,73,1


In [8]:
from scipy import sparse
from sklearn.preprocessing import OneHotEncoder, StandardScaler

ohe = OneHotEncoder()
scaller = StandardScaler()

In [9]:
def feature_engineering(df: pd.DataFrame) -> pd.DataFrame:
    #интеракции 'zone_id', 'campaign_clicks', 'country_id', 'os_id', 'impressions', 'banner_id'
    df['zone_id_banner_id'] = df['zone_id'].astype('str') + '_' + df['banner_id'].astype('str')
    df['zone_id_country_id'] = df['zone_id'].astype('str') + '_' + df['country_id'].astype('str')
    df['zone_id_os_id'] = df['zone_id'].astype('str') + '_' + df['os_id'].astype('str')
    df['country_id_os_id'] = df['country_id'].astype('str') + '_' + df['os_id'].astype('str')
    df['country_id_banner_id'] = df['country_id'].astype('str') + '_' + df['banner_id'].astype('str')
    df['os_id_banner_id'] = df['os_id'].astype('str') + '_' + df['banner_id'].astype('str')
    
    return df

In [10]:
test_banner_id1 = test.copy()
test_banner_id1['banner_id'] = test_banner_id1['banner_id1']

df = pd.concat([df, test_banner_id1], axis=0, ignore_index=True)

In [11]:
%%time
df = feature_engineering(df)
df.head()

CPU times: user 1min 50s, sys: 6.99 s, total: 1min 57s
Wall time: 1min 58s


,date_time,zone_id,banner_id,oaid_hash,campaign_clicks,os_id,country_id,banner_id0,rate0,g0,...,coeff_sum1,impressions,clicks,date,zone_id_banner_id,zone_id_country_id,zone_id_os_id,country_id_os_id,country_id_banner_id,os_id_banner_id
0,2021-09-01 00:02:49,30,596,5236744527665721365,0,0,7,596,0.050,0.064560,...,-4.370191,1,0,2021-09-01,30_596,30_7,30_0,7_0,7_596,0_596
1,2021-09-26 00:00:00,41,29,1834033519797437404,1,3,0,29,0.002,0.016386,...,-4.898257,1,0,2021-09-26,41_29,41_0,41_3,0_3,0_29,3_29
2,2021-09-26 00:00:00,1,188,7416450538971744701,2,2,15,188,0.008,0.014186,...,-2.106896,1,0,2021-09-26,1_188,1_15,1_2,15_2,15_188,2_188
3,2021-09-26 00:00:00,17,52,1832228443297591417,2,2,5,52,0.008,0.013550,...,-3.739501,1,0,2021-09-26,17_52,17_5,17_2,5_2,5_52,2_52
4,2021-09-26 00:00:00,47,73,4180077124914749282,1,4,13,73,0.008,0.120974,...,-3.037939,1,0,2021-09-26,47_73,47_13,47_4,13_4,13_73,4_73


In [12]:
%%time
data = sparse.hstack([
    #оригинальные фичи
    ohe.fit_transform(df[['zone_id']]),
    ohe.fit_transform(df[['os_id']]),
    ohe.fit_transform(df[['banner_id']]),
    ohe.fit_transform(df[['country_id']]),
    sparse.csr_matrix(scaller.fit_transform(df[['campaign_clicks']])),
    #интеракции
    ohe.fit_transform(df[['zone_id_banner_id']]),
    ohe.fit_transform(df[['zone_id_country_id']]),
    ohe.fit_transform(df[['zone_id_os_id']]),
    ohe.fit_transform(df[['country_id_os_id']]),
    ohe.fit_transform(df[['country_id_banner_id']]),
    ohe.fit_transform(df[['os_id_banner_id']])
])

CPU times: user 47.7 s, sys: 8.89 s, total: 56.6 s
Wall time: 58 s


In [13]:
data_train = data.tocsr()[:train_slice]
data_val = data.tocsr()[train_slice:train_slice+val_slice]
data_test = data.tocsr()[train_slice+val_slice:]

In [14]:
from sklearn.linear_model import LogisticRegression

In [15]:
%%time
clf_liblinear_final = LogisticRegression(random_state=0, 
                                     solver='liblinear',
                                     C=0.1, verbose=1
                                     ).fit(data.tocsr()[:train_slice+val_slice], df[df.date <= datetime.strptime('2021-10-01', '%Y-%m-%d').date()]['clicks'])


[LibLinear]iter  1 act 6.847e+05 pre 6.084e+05 delta 1.747e+00 f 9.491e+05 |g| 7.963e+05 CG   1
iter  2 act 8.521e+04 pre 7.061e+04 delta 1.747e+00 f 2.644e+05 |g| 1.809e+05 CG   1
cg reaches trust region boundary
iter  3 act 2.459e+04 pre 2.080e+04 delta 2.163e+00 f 1.792e+05 |g| 5.417e+04 CG   4
cg reaches trust region boundary
iter  4 act 6.677e+03 pre 5.832e+03 delta 2.607e+00 f 1.546e+05 |g| 1.776e+04 CG   4
cg reaches trust region boundary
iter  5 act 2.364e+03 pre 2.229e+03 delta 2.814e+00 f 1.479e+05 |g| 5.650e+03 CG   5
cg reaches trust region boundary
iter  6 act 1.645e+03 pre 1.614e+03 delta 3.125e+00 f 1.456e+05 |g| 3.118e+03 CG   6
cg reaches trust region boundary
iter  7 act 9.844e+02 pre 9.925e+02 delta 3.125e+00 f 1.439e+05 |g| 1.942e+03 CG   7
cg reaches trust region boundary
iter  8 act 7.681e+02 pre 7.693e+02 delta 3.200e+00 f 1.429e+05 |g| 1.509e+03 CG   9
cg reaches trust region boundary
iter  9 act 5.330e+02 pre 5.317e+02 delta 3.371e+00 f 1.422e+05 |g| 1.162e+03 

In [23]:
pred_proba = clf_liblinear_final.predict_proba(data_test[:test_slice])
print ('log_loss на тесте: ', log_loss(test.clicks, pred_proba))

log_loss на тесте:  0.1304476979828665


In [24]:
pred_proba_banner_id1 = clf_liblinear_final.predict_proba(data_test[test_slice:])
print ('log_loss на тесте (banner_id1): ', log_loss(test.clicks, pred_proba_banner_id1))

log_loss на тесте (banner_id1):  0.1366544531245715


## Считаем OPE метрику clipped ips на последнем дне 

In [ ]:
ope_df = test.copy()
ope_df['banner_id0_proba'] = pred_proba[:,1]
ope_df['banner_id1_proba'] = pred_proba_banner_id1[:,1]

ope_df['banner_id0_proba_logit'] = logit(ope_df['banner_id0_proba'])
ope_df['banner_id1_proba_logit'] = logit(ope_df['banner_id1_proba'])

ope_df['p0'] = 1 - norm.cdf(
    (- ope_df['coeff_sum0'] + ope_df['coeff_sum1'])/(np.abs(ope_df['g0']**2 + ope_df['g1']**2)**0.5)  
)

ope_df['p1'] = 1 - norm.cdf(
    (- ope_df['banner_id0_proba_logit'] + ope_df['banner_id1_proba_logit'])/(np.abs(ope_df['g0']**2 + ope_df['g1']**2)**0.5)
)

ope_df['ratio'] = ope_df['p1']/ope_df['p0']
ope_df['lambda'] = 10

### Важный момент. Те предикты которые у вас уже имеются, они работают для banner_id0, потому что он совпадает с banner_id. 
На самом деле не всегда. Поэтому мы оставим только те случаи, где совпадает. `fitted`

In [49]:
test[test.banner_id == test.banner_id0].shape

(1890562, 18)

In [50]:
test[test.banner_id != test.banner_id0].shape

(238416, 18)

In [54]:
fitted = ope_df[ope_df['banner_id'] == ope_df['banner_id0']]
cips = (fitted['clicks']*fitted[['ratio', 'lambda']].min(axis=1)).sum()/fitted.shape[0]

In [55]:
cips

0.0803272610351773

# Ответ:
0.080327